# Minimal Dispatch Example with oemof-tabular

The example models a simple electricity system as a 1-node-model the following nodes (buses and components):

* Bus 
* Volatile
* Storage 
* Dispatchable
* Load

In addition we will use a *excess* component for times of over-supply.

## Mathematical Formulation

Decision variable(s): $x \geq 0 $, (exogenous) parameter(s) $c$

#### Load constraint 

$$ x_{n,l}(t) = c^{load}_{l}(t) \qquad \forall t \in T$$

#### Dispatchable Generator constraint

$$x_{g,n}(t) \leq c^{max}_{g} \qquad \forall t \in T$$

#### Volatile Generator constraint 

$$x_{v,n}(t) = c^{max}_{v}(t) \qquad \forall t \in T$$

#### Storage constraints 

$$ x^{level}_{s}(t) = x^{level}_{s}(t) + x^{p}_{b, s} - x^{p}_{s, b}(t) \qquad \forall t \in T \ \{0, t_e\}$$

$$ x^{level}_s(t) \leq c_s^{max,level}$$ 

$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot c_s^{max,level}$$

#### Balance 

With the set of all Hub $H$: 

$$\sum_i x_{i, h}(t) - \sum_o x_{h, o}(t) - x^{excess}_h(t) = 0 \qquad \forall h \in H$$

#### Objective function

$$ \text{min:} \sum_t c^{cost}_g \cdot x_{g,h}(t)$$


## Python Imports

Imports of necessary python packages

In [ ]:
import os
import pkg_resources as pkg
import pandas as pd 

from oemof.solph import EnergySystem, Model, Bus
import oemof.tabular.tools.postprocessing as pp
import oemof.tabular.facades as fc

## Creating and setting the datapaths

Setting the datapath for raw-data and results

In [ ]:
# datapath for input data from the oemof tabular pacakge
datapath = pkg.resource_filename("oemof.tabular", "examples/data/data.xls")

# results path for output
results_path = os.path.join(
    os.path.expanduser("~"), "oemof-results", "dispatch", "output"
)

if not os.path.exists(results_path):
    os.makedirs(results_path)

Reading the prepared input data

In [ ]:
timeseries = pd.read_excel(
    datapath, sheet_name="timeseries", index_col=[0], parse_dates=True
)
timeseries.index.freq = "1H"

## Creating the model

In [ ]:
es = EnergySystem(timeindex=timeseries.index)

bus = Bus(label="DE")
es.add(bus)

es.add(
    fc.Dispatchable(
        label="ccgt",
        bus=bus,
        carrier="gas",
        tech="ccgt",
        capacity=100,
        marginal_cost=25,
    )
)

es.add(fc.Load(label="load", bus=bus, amount=500e3, profile=timeseries["load"]))

es.add(fc.Excess(label="excess", bus=bus))

es.add(
    fc.Volatile(
        label="wind",
        carrier="wind",
        tech="onshore",
        capacity=150,
        bus=bus,
        profile=timeseries["onshore"],
    )
)

es.add(
    fc.Storage(
        label="storage",
        bus=bus,
        carrier="lithium",
        tech="battery",
        capacity=20,
        marginal_cost=0.0001,
        balanced=True,
        initial_storage_capacity=0.5,
        storage_capacity=100,
    )
)

## Solving the model

In [ ]:
# create model based on energy system and its components
m = Model(es)

#  solve the model using cbc solver
m.solve("cbc")

# write results back to the model object
m.results = m.results()

# writing results with the standard oemof-tabular output formatt
pp.write_results(m, results_path)

print("Optimization done. Results are in {}.".format(results_path))


## Plotting the results

In [ ]:
from plotly import offline, plotly
from oemof.tabular.tools.plots import hourly_plot
offline.init_notebook_mode()

name = "dispatch"

# results path for output
results_path = os.path.join(os.path.expanduser("~"), "oemof-results", name, "output")

offline.plot(
    hourly_plot(
        name,
        "DE",
        os.path.join(os.path.expanduser("~"), "oemof-results"),
        plot_filling_levels=True,
    ),
    filename=os.path.join(results_path, "hourly-plot.html"),
)